# Basic Timing Evaluation
The following notebook demonstrates a workflow which evaluates the timing of a single single simulation run. The notebook directly accesses several Python classes and definitions which were provided by [FMITerminalBlock](https://github.com/AIT-IES/FMITerminalBlock) to process timing files. The main FMITerminalBlock executable may be configured to output these timing files in order to assess the real time performance of a simulation run.
The project specific python code is located in the ```timing``` package. The ```timing.data_set``` module defines a ```DataSet``` class and several utility classes which hold the timing data in the memory and provide some evaluation functions. Please consider the source code documentation for a more detailed description of accessed functions.

In [1]:
from timing import data_set as tds

## Load the Timing File
Timing files may be loaded via the convenience function ```tds.load_data_set_from_file(filename)```. The function constructs all facilities which are necessary to parse the timing file and to instantiate a ```tds.DataSet``` object. Please adjust the filename variable such that it points to the timing file which should be analyzed.

In [2]:
timing_file_path = r"S:\output-data\timing.csv" # <-- Adjust your path here
ds = tds.load_data_set_from_file(timing_file_path)

## Query Performance Indicators
FMITerminalBlock records the current real time instant and the current simulation time at various points during simulation. For instance, it is recorded when FMITerminalBlock predicts the next event and when the event is actually distributed to connected devices. In particular, the following time stamps are recorded:
* Registration time of events. The registration is the point in time where an event is predicted by the solver or sent to FMITerminalBlock by an external entity.
* Begin of distribution. During distribution the event is sent to all network devices and will be deleted afterwards
* End of distribution.

These numbers will be used to calculate certain performance metrics. During standard real time operation, FMITerminalBlock should start the distribution when the simulation time is reached in real time. The delay of predicted events at the begin of the distribution phase indicates that FMITerminalBlock is not able to complete all necessary operations before the event should be scheduled.

Since many metrics such as average delay can be applied to many processing stages and subsets of the timing data, ```tds.TimingAxis``` class was created. Any instance of ```tds.TimingAxis``` allows to query certain performance metrics for a subset of the timing data. ```tds.DataSet``` offers several functions which expose various ```tds.TimingAxis``` objects. The following sections will demonstrate most of the timing axis and performance indicators

### Direct Query
In order to query parameters regarding one axis (e.g. the registration time), the appropriate ```tds.TimingAxis``` object has to be queried.

In [3]:
registration_axis = ds.get_registration_axis()

The axis object may then be used to calculate certain timing statistics. For instance, use ```get_mean_delay()```, ```get_min_delay()```, ```get_max_delay()``` to query the mean, average, and maximum delay respectively. The variance of delay values may be obtained via ```get_variance_of_delay()```. Pleas note that per default the function also include early events. For instance predicted events are normally registered earlier than their associated time. Hence, their delay will be negative which also affects the statistical values. Additionally, the number of elements which are covered by a particular axis may be queried by ```get_length()```.

In [4]:
print("Number of registered timing samples: ", registration_axis.get_length())
print("Minimum delay:     ", registration_axis.get_min_delay())
print("Maximum delay:     ", registration_axis.get_max_delay())
print("Average delay:     ", registration_axis.get_mean_delay())
print("Variance in delay: ", registration_axis.get_variance_of_delay())

Number of registered timing samples:  30
Minimum delay:      2.42405
Maximum delay:      3.12285
Average delay:      2.77449
Variance in delay:  0.0444521638621


### Supported Axis

The following timing axes are supported. Each of the axis may be queried by the given function from any DataSource object. 
* ```get_registration_axis()```
* ```get_begin_distribution_axis()```
* ```get_end_distribution_axis()```

### Removal of Outliers 

The evaluation scripts support a basic mechanism which removes outliers. An outlier is a value with an exceptionally high or low delay value. Each timing axis object provides a function, ```get_delay_cleaned_axis()``` which removes the samples with the highest and lowest delay values and returns a newly constructed timing axis. The optional parameter ```outlier_factor``` specifies the share of removed samples. Per default 5% of the samples are removed. Please note that outliers are always removed symmetrically from the highest and lowest part of the delay spectrum.

In [5]:
clean_registration_axis = registration_axis.get_delay_cleaned_axis(outlier_factor=0.1)
print(clean_registration_axis.get_length(), " of ", registration_axis.get_length(), " samples left")

28  of  30  samples left


Satistics of each cleaned axis may be obtained as usual.

In [6]:
print("Number of registered timing samples: ", clean_registration_axis.get_length())
print("Minimum delay:     ", clean_registration_axis.get_min_delay())
print("Maximum delay:     ", clean_registration_axis.get_max_delay())
print("Average delay:     ", clean_registration_axis.get_mean_delay())
print("Variance in delay: ", clean_registration_axis.get_variance_of_delay())

Number of registered timing samples:  28
Minimum delay:      2.45245
Maximum delay:      3.09445
Average delay:      2.77456428571
Variance in delay:  0.0387018412698
